# Spread of an infectious disease
<font color='red'>Start your report with a general introduction to the topic.</font>

In the following, we will first explain the meaning of each variable. Then we will take a closer look at all the functions used to simulate the spreading. Next, we will explain how we have animated the simulation and finally we will show examples of how the simulation can be used properly.

Explanation of the variable names:

    c:              Choice of the user which simulations he wants
    n:              Number of rows in the dormitory
    m:              Number of columns in the dormitory
    initial_bed:    Row and column of the initial bed with the disease
    p:              Probability of an infection, between 0 and 1
    k:              Duration of the illness in days
    repetitions:    The number of days you want to simulate
    v:              Number of people vaccinated per day
    bs:             Number of beds swapped per day
    incu:           Duration of the incubation period in days 

## Cellular Automata
In this section we will take a closer look at the different functions we use to simulate propagation. 
Each field in the grid can have a different value indicating the current status. The following statuses are used:

    0:          Healthy
    1:          Healed
    -1:         Vaccinated
    > 1:        Sick
    > 1 + k:    Sick but not infectious (Incubation period)


First we initialize the grid and the desired size and set all fields to 0. Then we set the desired initial bed to sick and return the new grid.

In [5]:
import numpy as np

def initialize_grid(n, m, initial_bed, k, incu):
    grid = np.zeros((n, m))
    row, col = initial_bed
    grid[row, col] = 1+k+incu
    return grid

Next we have a function to find and return all neighbors of a bed. (top, bottom, left, right)

In [6]:
def get_neighbors(grid, row, col):
    n, m = grid.shape
    neighbors = []
    if row > 0:
        neighbors.append((row - 1, col))  # Top neighbor
    if row < n - 1:
        neighbors.append((row + 1, col))  # Bottom neighbor
    if col > 0:
        neighbors.append((row, col - 1))  # Left neighbor
    if col < m - 1:
        neighbors.append((row, col + 1))  # Right neighbor
    return neighbors

Now we start with the normal method to simulate the spread on one day. We show the evolution of this function and will add more features to the function with each step. As parameters we get the old grid, the probability of infection and the duration of the disease.
First we ask for the size of the grid and make a copy. Then we go through each field individually using two for loops. Then we check if the current field is sick (value greater than 1), if yes we subtract one sick day and search for all neighbors. For each neighbor it is checked if it is still healthy (value is 0) and then a random number is generated. If the neighbor is healthy and the generated number is greater than the value p, the neighbor is infected (value 1 + duration of the disease). Finally the new grid is returned.

In [7]:
def simulate_spread_basic(grid, p, k):
    n, m = grid.shape
    new_grid = np.copy(grid)
    for i in range(n):
        for j in range(m):
            if grid[i, j] > 1:  # Check if the bed is infected or already recovered
                new_grid[i,j] -= 1               
                for neighbor in get_neighbors(grid, i, j):
                    if grid[neighbor] == 0 and np.random.rand() < p:
                        new_grid[neighbor] = 1+k #set the neighbor ill                
    return new_grid

Now we will add a function that can vaccinate a certain number of healthy people every day. After a vaccination, the respective bed can no longer be infected. The function has two parameters, the number of people to vaccinate and the grid. 
A while loop is used to generate random numbers and try to vaccinate them. When enough vaccinations have been distributed, the new grid is returned.

In [9]:
def vaccinate(v, new_grid):
    n, m = new_grid.shape
    vaccinated = 0
    while(vaccinated < v):
        randi = np.random.randint(0,n-1)
        randj = np.random.randint(0, m-1)
        if new_grid[randi, randj] == 0:
            new_grid[randi, randj] = -1
            vaccinated += 1

    return new_grid

Now we will add a function that will swap a certain number of beds with each other every day. The function has two parameters, the number of beds to swap and the grid. 
A While loop is used to generate random numbers to select two beds. Then the values of the two beds are exchanged. When enough beds have been swapped, the new grid is returned.

In [10]:
def bed_swap(bs, new_grid):
    n, m = new_grid.shape
    bedsSwaped = 0
    while(bedsSwaped < bs):
        randi1 = np.random.randint(0,n-1)
        randj1 = np.random.randint(0, m-1)
        randi2 = np.random.randint(0,n-1)
        randj2 = np.random.randint(0, m-1)
        tmpVal = new_grid[randi1, randj1]
        new_grid[randi1, randj1] = new_grid[randi2, randj2]
        new_grid[randi2, randj2] = tmpVal
        bedsSwaped += 1

    return new_grid

Ganze Funktion

The first part of the function remains almost the same, with the difference that it counts how many healthy beds are left. 
Then it checks if there are enough healthy beds left to vaccinate. If not, the value of beds to vaccinate is decreased.